---
# Ziel


In diesem Notebook wird der Ablauf zum Erstellen und Trainieren eines Neuronalen Netzes mit Keras gezeigt. 


---

# 1. Allgemeine Vorbereitung

Zu Beginn wird in den Ordner "KI" navigiert, der in der vorangegangenen Klickanleitung erstellt wurde. 

Dafür wird der Befehl cd (= change directory) verwendet. 

In [1]:
%cd ~/../content/drive/My\ Drive/KI/


/content/drive/My Drive/KI


In Google Colab sind viele Libraries (Bibliotheken) bereits vorinstalliert. Um die Funktionen dieser Libraries zu nutzen, werden sie mit dem import-Befehl in das aktuelle Notebook importiert.

Zum Ausführen dieses Notebooks werden die drei Bibliotheken keras, matplotlib und numpy benötigt, die folgende Funktionen erfüllen:


* keras: Funktionen für das Neuronale Netz

* matplotlib: Zum Anzeigen von Bildern und Plotten von Funktionen

* numpy: Zur numerischen Berechnung von Arrays



In [2]:
import keras
from keras.callbacks import ModelCheckpoint 
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import matplotlib
from matplotlib import pyplot as plt

import numpy as np




---



# 2. Erstellen des Datensets

In Github befindet sich ein Datenset mit Bildern von Kreisen, Dreiecken und Vierecken. Dieses Datenset wird mit folgendem Befehl heruntergeladen.

In [3]:
!git clone https://github.com/FeliziaTUDo/Formen.git

Cloning into 'Formen'...
remote: Enumerating objects: 1541, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 1541 (delta 21), reused 0 (delta 0), pack-reused 1452
Receiving objects: 100% (1541/1541), 1.07 MiB | 3.92 MiB/s, done.
Resolving deltas: 100% (61/61), done.
Checking out files: 100% (1713/1713), done.


Aufgabe 1

(Die Lösung zu allen Aufgaben befindet sich am Ende dieses Notebooks unter der Überschrift "Lösungen zu den Aufgaben")

1a.) Öffne die Ordnerstruktur am linken Rand des Notebooks. In welchem Verzeichnis wurden die Bilder gespeichert? 

1b.) Nach welchem System wurden die Bilder sortiert? Weshalb ist diese Sortierung notwendig? 

1c.) Um die Trainingsbilder und Testbildern in ein Datenset einzufügen, wird der Pfad dorthin benötigt. In der nächsten Zelle wird der Pfad für die Trainignsbilder mit der Variable path_train_images angegeben. Gib nach dem gleichen Schema den den Pfad für die Testbilder unter der Variable path_test_images an.

In [4]:
path_train_images = './Formen/train'
path_test_images = '.'

Im nächsten Schritt werden die Bilder in einem Datenset angeordnet. Zu jedem Bild wird die entsprechende Klasse (= Label) gespeichert. 

In unserem Datenset sind alle Bilder bereits in der gleichen Größe. In anderen Datensets kann es der Fall sein, dass Bilder unterschiedlich groß sind. Diese Bilder werden bei Anordnung im Datenset auf die vorgegebene Größe verkleinert. 

Daher wird zu Begin die Seitenlänge des Bildes in Pixeln in der Variablen image_side_lentgh festgelegt. In diesem Fall beträgt die Seitenlänge des Bildes 28 Pixel. 

Aufgabe 2: Warum müssen alle Bilder die gleiche Seitenlänge haben?

In [5]:
# Welche Seitenlänge in Pixeln sollen die Bilder haben?
image_side_lentgh = 

In der nächsten Zelle werden die Bilder in ein Datenset zusammengefügt. 

Mit ImageDataGenerator() werden einzelne Pakete ("Batches") aus Bilddaten erzeugt. 

"rescale" ist der Faktor, mit dem alle eingehenden Daten multipliziert werden. Die Helligkeitswerte befinden sich in Werten von 0 (schwarz) bis 255 (weiß). Um diese auf Werte zwischen 0 und 1 zu normieren, wird der unten gezeigte rescale-Faktor angewendet.

"validation_split" gibt an, wie groß der Anteil der Bilder ist, die nicht für den Lernprozess verwendet werden, sondern mit denen die Qualität der Gewichte des Netzes überprüft wird.  

Weitere Informationen zum ImageDataGenerator befinden sich hier: https://keras.io/api/preprocessing/image/

In [6]:
# Einstellung zum Einlesen der Bilder
data_generation = ImageDataGenerator(rescale = 1./ 255,
                                     validation_split=0.20)

Ein Trainingsdatenset und ein Validationsdatenset werden erstellt. 

Hier wird der Pfad zu den Trainingsbildern und die Seitenlänge der Bilder angegeben. Über die Variable batch_size wird die Anzahl der Bilder festgelegt, nach der die Gewichte des Neuronalen Netzes aktualisiert werden. Mit class_mode wird die Art des Abspeicherns der Labels festgelegt.  

In [ ]:
## Trainingsdatenset
train_images = data_generation.flow_from_directory( path_train_images,
                                                    target_size = (image_side_lentgh,image_side_lentgh),
                                                    batch_size = 32,
                                                    seed = 100,
                                                    class_mode = 'categorical',
                                                    color_mode = 'grayscale',
                                                    subset='training'
                                                    )

## Validationdatenset
validation_images = data_generation.flow_from_directory(path_train_images,
                                                    target_size=(image_side_lentgh,image_side_lentgh),
                                                    batch_size = 32,
                                                    seed = 100,
                                                    class_mode='categorical',
                                                    color_mode = 'grayscale',
                                                    subset='validation')

---

# 3. Aufbau des Neuronalen Netzes

In diesem Kapitel wird ein Neuronales Netz aus drei Schichten aufgebaut. 

Zunächst wird die Anzahl der Klassen festgelegt. 

Aufgabe 3

3a.) Wie viele Klassen sind vorhanden und wie kommt diese Zahl zustande?

3b.) Welche Schicht des Neuronalen Netzes wird durch die Anzahl der Klassen festgelegt?

In [9]:
# Wie viele Klassen haben wir? 
num_classes = 

Die Anzahl der Neuronen in der verdeckten, zweiten Schicht (Hidden Layer) werden durch keinen Parameter vorgegeben. Wir verwenden hier 10 Neuronen.

In [10]:
neurons_second_layer = 

Im nächsten Schitt wird das Neuronale Netz aufgebaut. 

Mit der Angabe Sequential() wird festgelegt, dass Schichten nacheinander über model.add() hinzugefügt werden. 

Zwei Arten von Schichten werden verwendet: Dense und Flatten. Dense bedeutet, dass eine eindimensinale Schicht vorliegt, deren Neuronen vollständig mit allen Neuronen der nachfolgenden Schicht verknüpft sind.
Die Schicht Flatten gibt ebenfalls eine eindimensionale Schicht Neuronen aus. Zusätzlich erfüllt sie eine weitere Funktion: 

Aufgabe 4: Welche Funktion könnte die Schicht "Flatten" erfüllen, und warum ist diese Funktion notwendig?

Bei Hinzufügen einer Schicht werden die Anzahl der Neuronen durch die Variable "units" festgelegt. Ausnahme stellt die erste Schicht dar, in der die Anzahl der Neuronen über input_shape festgelegt werden. 

Bei allen Schichten (außer der ersten Schicht) wird eine Funktion angegeben, nach der die Neuronen aktiviert werden (Aktivierungsfunktion). Informationen zu möglichen Aktivierungsfunktionen befinden sich bspw. hier:

 https://ml-cheatsheet.readthedocs.io/en/latest/activation_functions.html



In [13]:
# Initialisierung des Neuronalen Netzes
model = Sequential()

# Aufbau des Neuronalen Netzes
## Schicht 1:
model.add(Flatten(input_shape=(image_side_lentgh,image_side_lentgh, 1)))

## Schicht 2:
model.add(Dense(units=neurons_second_layer, activation='sigmoid'))

## Schicht 3:
model.add(Dense(units=num_classes, activation='softmax'))

In der nachfolgenden Tabelle wird eine Übersicht des neuronalen Netzes ausgegeben. 


In [14]:
## Ausgabe der Netzarchitektur  
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                7850      
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 33        
Total params: 7,883
Trainable params: 7,883
Non-trainable params: 0
_________________________________________________________________


In der nächsten Zelle wird das Neuronale Netz kompiliert (= in Maschinencode umgewandelt). 

Mit dem Optimizer werden Einstellungen für die Anpassung der Gewichte festgelegt. Die Lernrate (learning_rate) bestimmt beispielsweise die Schrittgröße bei der Gewichtsanpassung. Der Parameter "loss", beschreibt im Groben die Abweichung der Gewichte der Output-Layer von den idealen Gewichten der Output-Layer. Er wird häufig auch als Kostenfunktion bezeichnet. Es gibt verschiedene Möglichkeiten, des Loss zu berechnen. Weitere Informationen zu Loss-Funktionen findest du hier: 

https://keras.io/api/losses/probabilistic_losses/#categoricalcrossentropy-class

Mit dem Parameter "metrics" das Verfahren zur Beurteilung der Ergebnisse festgelegt. In dem Fall beschreibt "Accuracy" das Verhältnis der richtig vorhergesagten Bildern zu allen vorhergesagten Bildern. Dieses Verhältnis liegt immer zwischen 0 und 1. 0 bedeutet, dass kein einziges Bild korrekt vorhergesagt wurde, 1 bedeutet, dass alle Bilder korrekt vorhergesagt wurden.

Aufgabe 2e: Sollte der Loss eher minimiert oder maximiert werden? 
Aufgabe 2f: Sollte die Accuracy eher minimiert oder maximiert werden?
Aufgabe 2g: In diesem Fall wird das Neuronale Netz auf drei Klassen trainiert. Welche Accuracy ist bei einem vollständig untrainierten Netz zu erwarten?

Zusätzlich wird ein checkpoint angelegt, d.h. ein Speicherort für die besten Gewichtungen, die mithilfe des Validierungsdatensets ermittelt werden. 


In [16]:
# Kompilieren des Neuronalen Netzes
my_optimizer = Adam(learning_rate = 0.001) 
model.compile(optimizer = my_optimizer,
                   loss = 'categorical_crossentropy',
                   metrics = 'accuracy')   

## Vorgabe zum Speichern des Netzes
checkpoint_path = "Checkpoint_NeuronalesNetz.ckpt"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    verbose = 1,
    mode='max',
    save_best_only=True)               

# 4. Training des Neuronalen Netzes

Das Neuronale Netz wird trainiert. Die Trainingsbilder und Validierungsbilder werden angegeben, sowie die Methode (callbacks), nach der die besten Gewichtungen gespeichert werden. 

Eine Epoche beschreibt den Ablauf, in dem jedes Bild des Datensets ein mal das Neuronale Netz durchlaufen hat. 

Nach einem Batch wird der Loss und die Accuracy geupdatet.

In [ ]:
# Training des Neuronalen Netzes
history = model.fit(train_images,
                    epochs = 100,
                    validation_data=validation_images,
                    callbacks = model_checkpoint_callback
                    )                    

# 5. Treffen von Vorhersagen

Zunächst werden die besten Gewichtungen geladen. Die 30 Testbilder werden mit dem ImageDataGenerator genauso bearbeitet, wie die Trainingsbilder. 

In [ ]:
# Laden der Gewichtungen des Neuronalen Netzes
model.load_weights(checkpoint_path)

# Vorhersage des Bildes
image_index = 0
number_of_images = 30
test_generator = ImageDataGenerator(rescale = 1./ 255)

test_images = data_generation.flow_from_directory( path_test_images,
                                                   target_size = (image_side_lentgh,image_side_lentgh),
                                                   class_mode = 'categorical',
                                                   color_mode = 'grayscale',
                                                   shuffle= False
                                                   )


Die Testbilder werden durch den Befehl predict vorhergesagt. Alle Bilder werden im Anschluss mit der dazugehörigen Vorhersage angezeigt. 

Aufgabe 6

6a.) Bei welchen Formen funktioniert die Vorhersage gut, bei welchen Formen funktioniert die Vorhersage weniger gut? 

6b.) Was könnte der Grund dafür sein, dass manche Formen leichter vorhergesagt werden, als andere? Tipp: Betrachte dafür auch die Trainingsbilder.

In [ ]:
all_predictions = model.predict(test_images)

# Anzeige der vorhergesagten Bilder
x,y = test_images.next()

for i in range(image_index ,image_index+number_of_images):
    one_pred = all_predictions[i]
    print(one_pred)

    predicition_name = np.argmax(one_pred)
    if predicition_name == 0:
        print("Kreis") 
    if predicition_name == 1:
        print("Viereck")
    if predicition_name == 2:
        print("Dreieck")

    image = x[i]
    print(image.shape)
    plt.imshow(image.reshape(28,28), cmap = "gray")
    plt.show()

Zum Überprüfen der Qualität des Neuronalen Netzes werden die Loss-Funktion und die Accuracy der Validierungsdaten der Trainings- und der Testbilder aufgezeichnet. 

In [ ]:
# Plotten der Genauigkeit (Accuracy)
plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train' ], loc='upper left')
plt.show()

plt.plot(history.history['val_accuracy'], label='Categorical Crossentropy (training data)')
plt.title('Validation Accuracy')
plt.ylabel('categorical_crossentropy value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Plotten der Loss-Funktion
plt.plot(history.history['loss'], label='Categorical Crossentropy (training data)')
plt.title('Model loss')
plt.ylabel('categorical_crossentropy value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

plt.plot(history.history['val_loss'], label='Categorical Crossentropy (training data)')
plt.title('Val_Loss')
plt.ylabel('categorical_crossentropy value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

# Weitere Aufgaben zum Experimentieren mit dem Code





7a.) Setze die Anzahl der Epochen durch die Variable "epochs = " bei der Funktion model.fit() einmal auf 10 und lass das Neuronale Netz mit dieser Anzahl Epochen trainieren. Wenn du die darauf folgenden Zellen ausführst, in denen die Plots erstellt werden: Wie verändert sich der Plot der Accuracy und der Loss-Funktion bei sehr wenigen Epochen? 

7b.) Führe Aufgabe 9 mit 200 Epochen statt 10 Epochen aus.  

7c.) (schwierig) Kannst du zu beiden Fällen (Aufgabe 9 und 10) die Begriffe Overfitting und Underfitting zuorden?

8a.) Veränder die Eingabeneuronen von 28 auf 14 und lass das neuronale Netz erneut trainieren. Welches Ergebnis für die Accuracy erwartest du? 

8b.) Veränder die Eingabeneuronen von 28 auf 7 und lass das neuronale Netz erneut trainieren. Welches Ergebnis für die Accuracy erwartest du? 

8c.) Veränder die Eingabeneuronen von 28 auf 7 und lass das neuronale Netz erneut trainieren. Welches Ergebnis für die Accuracy erwartest du? 

9.) Veränder den Parameter batch_size im zweiten Abschnitt von 32 auf 64. Welche Auswirkung hat diese Veränderung auf die val_accuracy beim Training?

10.) Veränder die Lernrate (learning_rate) in model.fit() von 0.001 auf 0.005. Was für einen Effekt erwartest du auf die val_accuracy beim Training?

11.) Aktuell werden in der zweiten Dense-Layer 10 Neuronen verwendet. Verändere die Neuronen der 2. Schicht (Second Layer) und lass das neuronale Netz erneut trainieren (Dafür musst du alle Zellen ab dem Zeitpunkt erneut ausführen). Als Vorschlag: Teste die Anzahl 3 und 25. Wird das Ergebnis besser oder schlechter? 

12.) Setz in Abschnitt 3 in der letzten Zelle den Parameter save_best_only von True auf False. Was verändert sich im Training?

13.) Füg dem neuronalen Netz eine weitere Dense-Layer hinzu, die du nach der ersten Dense-Layer einfügst. Variiere auch hier die Anzahl der Neuronen (z.B. 5 oder 9). Was ändert sich im Training? 


---
# Lösungen zu den Aufgaben

1a.) Öffne die Ordnerstruktur am linken Rand des Notebooks. In welchem Verzeichnis wurden die Bilder gespeichert? 

Die Bilder befinden sich unter drive/My\ Drive/Formen/ 


1b.) Nach welchem System wurden die Bilder sortiert? Weshalb ist diese Sortierung notwendig? 

Die Bilder sind nach Trainings- und Testbildern sortiert. Mit den Trainingsbildern wird das Netz trainiert, mit den Testbildern wird die Qualität des Netzes getestet. 
Zusätzlich sind die Bilder nach der abgebildeten Form sortiert. Alle Kreise, alle Dreiecke und alle Vierecke für das Training befinden sich in je einem Ordner. Dadurch wird gewährleistet, dass den Bildern die richtige Klasse zugeordnet wird

1c.) Um die Trainingsbilder und Testbildern in ein Datenset einzufügen, wird der Pfad dorthin ebnötigt. In der nächsten Zelle wird der Pfad für die Trainignsbilder mit der Variable path_train_images angegeben. Gib nach dem gleichen Schema den den Pfad für die Testbilder unter der Variable path_test_images an.

path_test_images = './Formen/test'

2.) Warum müssen alle Bilder die gleiche Seitenlänge haben?

Jedes Pixel entspricht einem Neuron in der Input Layer des Neuronalen Netzes. Wenn sich die Größen der Bilder unterscheiden, sind entweder zu viele oder zu wenige Neuronen in dieser Schicht worhanden.

3a.) Wie viele Klassen sind vorhanden und wie kommt diese Zahl zustande?

Drei Klassensind vorhanden , da drei verschiedene Formen (Dreiecke, Kreise und Vierecke) unterschieden werden. 

3b.) Welche Schicht des Neuronalen Netzes wird durch die Anzahl der Klassen festgelegt?

Durch die Anzahl der Klassen wird die Anzahl der Ausgabeneuronen (Neuronen in der Output Layer) festgelegt. 

4) Welche Funtkion könnte die Schicht "Flatten" erfüllen, und warum ist diese Funktion notwendig?

Die Schicht Flatten erstellt aus einem zweidimensionalen Bild einen eindimensionalen Vektor. Dieser ist notwendig, um jedem Neuron der Input Layer einen Wert zuzuweisen. 

5.) In diesem Fall wird das Neuronale Netz auf drei Klassen trainiert. Welche Accuracy ist bei einem vollständig untrainierten Netz zu erwarten?

Ein untrainiertes Netz würde zufällig Vorhersagen treffen. Somit läge die erwartete Accuracy bei 1/3 (0.3333).

6a.) Bei welchen Formen funktioniert die Vorhersage gut, bei welchen Formen funktioniert die Vorhersage weniger gut? 

Bei Dreiecken und Vierecken funktioniert die Vorhersage deutlich besser als bei Kreisen.

6b.) Was könnte der Grund dafür sein, dass manche Formen leichter vorhergesagt werden, als andere? Tipp: Betrachte dafür auch die Trainingsbilder.

Dreiecke und Vierecke sind leichter voneinander abzugrenzen als Kreise, insbesondere wenn sie unsauber gezeichnet sind. Das ist auch im Datenset erkennbar, dass ein unsauber gezeichnetes Dreieck eher als Kreis, nicht aber als Viereck interpretierbar ist. Das gleiche gilt für Vierecke.  

7a.) Setze die Anzahl der Epochen durch die Variable "epochs = " bei der Funktion model.fit() einmal auf 10 und lass das Neuronale Netz mit dieser Anzahl Epochen trainieren. Wenn du die darauf folgenden Zellen ausführst, in denen die Plots erstellt werden: Wie verändert sich der Plot der Accuracy und der Loss-Funktion bei sehr wenigen Epochen? 
                    
Bei wenigen Epochen hat das Neuronale Netz noch nicht ausreichend trainiert. Der Loss ist noch vergleichsweise hoch, die Accuracy vergleichsweise niedrig. Das heißt, das Netz erkennt viele Bilder noch nicht.  

Folgender Code wird in für model.fit() verwendet: 

```
history = model.fit(train_images,
                    epochs = 10,
                    validation_data=validation_images,
                    callbacks = model_checkpoint_callback
                    )
```

         


7b.) Führe Aufgabe 9 mit 200 Epochen statt 10 Epochen aus.  

Bei vielen Epochen kann es sein, dass der Validierungsloss erneut ansteigt und die Validierungs-Accuracy sinkt. Das bedeutet, dass sich das Netz zu gut an die Trainingsbilder gewöhnt hat, und dadurch schlechter im Erkennen der Testbilder wird.    



7c.) (schwierig) Kannst du zu beiden Fällen die Begriffe Overfitting und Underfitting zuorden?

Underfitting bedeutet, dass das Neuronale Netz (noch) nicht genügend Muster aus dem Training herausfinden konnte. Dies kann durch ein zu kurzes Training resultieren. (Siehe Aufgabe 7a)

Overfitting bedeutet, dass das Neuronale Netz die Daten zu gut kennt und sie "auswendig gelernt" hat. Als Konsequenz kann das Neuronale Netz neue Bilder schlechter erkennen und somit verschlechtern sich die Validierungswerte. Dieser Effekt tritt bei einem zu langem Training auf. (Siehe Aufgabe 7b.) 

8a.) Veränder die Eingabeneuronen von 28 auf 14 und lass das neuronale Netz erneut trainieren. Welches Ergebnis für die Accuracy erwartest du? 

8b.) Veränder die Eingabeneuronen von 28 auf 7 und lass das neuronale Netz erneut trainieren. Welches Ergebnis für die Accuracy erwartest du? 

8c.) Veränder die Eingabeneuronen von 28 auf 7 und lass das neuronale Netz erneut trainieren. Welches Ergebnis für die Accuracy erwartest du? 

In 8a und 8b werden die Bildinformationen reduziert. Dem neuronalen Netz stehen weniger Informationen zum Anpassen der Parameter zur Verfügung. Wenn es zu wenig Parameter sind, wird eine geringere Accuracy und ein höherer Loss erreicht. 

Bei 8c werden die Pixel "vergrößert". Da das Originalbild nur 28 Pixel hat, liegt die Information jedes Pixels jetzt 4 Mal vor. Das heißt, das neuronale Netz muss jetzt deutlich mehr Paramter lernen. Es kann sein, dass es mit dieser Konfiguration mit den vorhandenen Neuronen der 2. Schicht und der Lernrate gar nicht lernt.

Dieser Code muss verändert werden: \\
Welche Seitenlänge in Pixeln sollen die Bilder haben? 

8a: image_side_lentgh = 14 \\
8b: image_side_lentgh = 7 \\
8c: image_side_lentgh = 56


9.) Veränder den Parameter batch_size im zweiten Abschnitt von 32 auf 64. Welche Auswirkung hat diese Veränderung auf die val_accuracy beim Training?

```
## Trainingsdatenset
train_images = data_generation.flow_from_directory( path_train_images,
                                                    target_size = (image_side_lentgh,image_side_lentgh),
                                                    batch_size = 32,
                                                    seed = 100,
                                                    class_mode = 'categorical',
                                                    color_mode = 'grayscale',
                                                    subset='training'
                                                    )

## Validationdatenset
validation_images = data_generation.flow_from_directory(path_train_images,
                                                    target_size=(image_side_lentgh,image_side_lentgh),
                                                    batch_size = 32,
                                                    seed = 100,
                                                    class_mode='categorical',
                                                    color_mode = 'grayscale',
                                                    subset='validation')
```
Durch die erhöhte Batch_size wird die das neuronale Netz weniger oft geupdatet. Dadurch sind weniger Sprünge vorhanden. 

10.) Veränder die Lernrate (learning_rate) in model.fit() von 0.001 auf 0.005. Was für einen Effekt erwartest du auf die val_accuracy beim Training?

Die Schrittweite wird erhöht. Wenn das Training funktioniert, geht es schneller als bei einer kleineren Schrittweite. Es kann aber auch sein, dass hier kein Minimum gefunden wird, da das Minimum durch die Schrittweite "übersprungen" wird. 

Code zum Anpassen: 
```
# Kompilieren des Neuronalen Netzes
my_optimizer = Adam(learning_rate = 0.005) 
model.compile(optimizer = my_optimizer,
                   loss = 'categorical_crossentropy', #'mse', ##'categorical_crossentropy',
                   metrics = 'accuracy')   
```

11.) Aktuell werden in der zweiten Dense-Layer 10 Neuronen verwendet. Verändere die Neuronen der 2. Schicht (Second Layer) und lass das neuronale Netz erneut trainieren (Dafür musst du alle Zellen ab dem Zeitpunkt erneut ausführen). Als Vorschlag: Teste die Anzahl 5 und 25. Wird das Ergebnis besser oder schlechter?

Code zum Anpassen (Beispiel für eine neue mittlere Schicht mit 3 Neuronen): 
``` 
neurons_second_layer = 5
```
Für 25 Neuronen: 
``` 
neurons_second_layer = 25
```
Wenn bei wenigen Neuronen die gleiche Accuracy erreicht wird, wie bei vielen Neuronen, bedeuet es, dass nur wenige Neuronen zum Erlernen der Merkmale notwendig sind. Wenn bei vielen Neuronen eine höhere Accuracy und ein geringerer Loss erreicht wird, bedeutet es, dass mit diesen Neuronen neue Merkmale erlernt werden. 

12.) Setz in Abschnitt 3 in der letzten Zelle den Parameter save_best_only von True auf False. Was verändert sich im Training?

Code zum Anpassen
```
## Vorgabe zum Speichern des Netzes
checkpoint_path = "Checkpoint_NeuronalesNetz.ckpt"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    verbose = 1,
    mode='max',
    save_best_only=False)  
```
Es wird nun immer das letzte Modell abgespeichert und nicht mehr das beste. 

13.) Füg dem neuronalen Netz eine weitere Dense-Layer hinzu, die du nach der ersten Dense-Layer einfügst. Variiere auch hier die Anzahl der Neuronen (z.B. 5 oder 9). Was ändert sich im Training? 


``` 
# Initialisierung des Neuronalen Netzes
model = Sequential()

# Aufbau des Neuronalen Netzes
## Schicht 1:
model.add(Flatten(input_shape=(image_side_lentgh,image_side_lentgh, 1)))

## Schicht 2:
model.add(Dense(units=neurons_second_layer, activation='sigmoid'))

## neue Schicht nach 2:
model.add(Dense(units=3, activation='sigmoid'))

## Schicht 3:
model.add(Dense(units=num_classes, activation='softmax'))
```

Wenn durch die neue Schicht neue Merkmale erlernt werden, steigt die Accuracy. Ansonsten sinkt sie. 

Quelle: https://medium.com/tebs-lab/how-to-classify-mnist-digits-with-different-neural-network-architectures-39c75a0f03e3
